# BEAST Analysis Notebook

---

# 0. SETUP

In [1]:
import os
import pandas as pd
import seaborn as sns
from Bio import Phylo, AlignIO
from functions import *

## Paths

In [2]:
#project_dir         = os.path.dirname(os.path.dirname(os.getcwd()))
#project_dir         = os.path.join(project_dir, "results")
project_dir          = "/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main"

#tree_dir            = project_dir + "/beast/all/chromosome/clade/summary_trees_noHyperPrior/relaxed_clock"
tree_dir            = project_dir + "/beast/all/chromosome/clade/phylogeography/"
metadata_path       = project_dir + "/iqtree/all/chromosome/full/filter5/filter-taxa/metadata.tsv"
beast_dir           = project_dir + "/beast/all/chromosome/clade"

auspice_config_path = project_dir + "/config/auspice_config.json"
out_path_colors     = project_dir + "/augur/all/chromosome/full/filter5/colors.tsv"
out_path_latlon     = project_dir + "/augur/all/chromosome/full/filter5/latlon.tsv"

# ------------------------------------------
# Alignment
constant_sites_path = project_dir + "/snippy_multi/all/chromosome/full/snippy-multi.constant_sites.txt"
aln_path            = project_dir + "/iqtree/all/chromosome/full/filter5/filter-sites/snippy-multi.snps.aln"


## Variables

In [3]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)

# ------------------------------------------
BRANCH_LIST = {
    "1.ORI" : ["1.ORI1", "1.ORI2", "1.ORI3"],
    "1.IN": ["1.IN1","1.IN2","1.IN3"],  
    "1.ANT": ["1.ANT1"], 
    "1.PRE" : ["1.PRE0","1.PRE1", "1.PRE2", "1.PRE3"],
    "2.MED": ["2.MED0", "2.MED1","2.MED2","2.MED3" ],      
    "2.ANT": ["2.ANT1","2.ANT2","2.ANT3" ],    
    "4.ANT": ["4.ANT1" ],       
    "3.ANT": ["3.ANT1", "3.ANT2" ],  
    "0.ANT": ["0.ANT1", "0.ANT2","0.ANT3","0.ANT5"],         
    "0.ANT4" : ["0.ANT4"], 
    "0.PE": ["0.PE2", "0.PE4m", "0.PE4m", "0.PE4t", "0.PE4a", "0.PE5", "0.PE7", "0.PE8", "0.PE10"],   
    "0.PRE": ["0.PRE1", "0.PRE2"],        
}

NUM_STATES = 10

NO_DATA_CHAR = "NA"
JSON_INDENT=2

# ------------------------------------------
# Alignment
with open(constant_sites_path) as infile:
    data = infile.read().strip().split(",")
    constant_sites = sum([int(count) for count in data])

aln = AlignIO.read(aln_path, "fasta")
variant_sites = len(aln[0].seq)
SEQ_LEN = constant_sites + variant_sites

POSTERIOR_THRESH = 95

---

# 1. IMPORT

## Metadata

In [4]:
metadata_df = pd.read_csv(metadata_path, sep='\t')
metadata_df.set_index(metadata_df.columns[0], inplace=True)
metadata_df.fillna(NO_DATA_CHAR, inplace=True)

display(metadata_df)

,strain,date,date_bp,country,province,country_lat,country_lon,province_lat,province_lon,biovar,branch_major,branch_minor,biosample_accession,biosample_comment,branch_number,continent,date_mean,date_bp_mean,date_err,lat,lon,host_human,branch_major_color,geometry_size,geometry,root_rtt_dist,clade_rtt_dist
sample,,,,,,,,,,,,,,,,,,,,,,,,,,,
Reference,CO92,1992,-29,United States of America,Colorado,39.783730,-100.445882,38.7252,-105.608,Orientalis,1.ORI,1.ORI1,SAMEA1705942,KEEP: Assembly Modern Reference,1,North America,1992.0,29.0,0.0,38.725178,-105.607716,Human,#ff0000,1.0,POINT (-105.607716 38.7251776),0.000073,0.000006
GCA_009909635.1_ASM990963v1_genomic,9_10,1923.0,-98,Russia,Rostov Oblast,64.686314,97.745306,47.6222,40.7958,Medievalis,2.MED,2.MED1,SAMN13632815,KEEP: Assembly Modern,2,Europe,1923.0,98.0,0.0,47.622245,40.795794,Human,#b3f396,4.0,POINT (40.7957942 47.6222451),0.000073,0.000010
GCA_009669545.1_ASM966954v1_genomic,42126,2006.0,-15,China,Xinjiang,35.000074,104.999927,42.4805,85.4633,Antiqua,0.ANT,0.ANT1,SAMN07722925,KEEP: Assembly Modern,0,Asia,2006.0,15.0,0.0,42.480495,85.463346,Non-Human,#1996f3,105.0,POINT (85.46334640000001 42.4804953),0.000054,0.000012
GCA_009669555.1_ASM966955v1_genomic,42123,2005.0,-16,China,Xinjiang,35.000074,104.999927,42.4805,85.4633,Antiqua,0.ANT,0.ANT1,SAMN07722924,KEEP: Assembly Modern,0,Asia,2005.0,16.0,0.0,42.480495,85.463346,Non-Human,#1996f3,105.0,POINT (85.46334640000001 42.4804953),0.000055,0.000012
GCA_009669565.1_ASM966956v1_genomic,42118,2005.0,-16,China,Xinjiang,35.000074,104.999927,42.4805,85.4633,Antiqua,0.ANT,0.ANT1,SAMN07722923,KEEP: Assembly Modern,0,Asia,2005.0,16.0,0.0,42.480495,85.463346,Non-Human,#1996f3,105.0,POINT (85.46334640000001 42.4804953),0.000055,0.000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMEA7313243_45,Azov38,[1400:1700],[-621:-321],Russia,Rostov Oblast,64.686314,97.745306,47.6222,40.7958,Second Pandemic,1.PRE,1.PRE1,SAMEA7313243_45,KEEP: SRA Ancient Combined Record,1,Europe,1550.0,471.0,150.0,47.622245,40.795794,Human,#e6ce74,4.0,POINT (40.7957942 47.6222451),0.000075,0.000022
SAMEA7313246_49,Gdansk8,[1400:1700],[-621:-321],Poland,Pomeranian Voivodeship,52.215933,19.134422,54.2456,18.1099,Second Pandemic,1.PRE,1.PRE1,SAMEA7313246_49,KEEP: SRA Ancient Combined Record,1,Europe,1550.0,471.0,150.0,54.245560,18.109900,Human,#e6ce74,1.0,POINT (18.1099 54.24556),0.000060,0.000007
SAMEA6651390,AGU010,[1435:1477],[-586:-544],Lithuania,Vilnius County,55.350000,23.750000,54.8227,25.2495,Second Pandemic,1.PRE,1.PRE1,SAMEA6651390,KEEP: SRA Ancient,1,Europe,1456.0,565.0,21.0,54.822692,25.249534,Human,#e6ce74,3.0,POINT (25.24953400167952 54.82269205),0.000060,0.000006


## Most Recent Sampling Date

In [5]:
out_path = os.path.join(beast_dir, "most_recent_sampling_dates.tsv")
mrsd_dict = {}

with open(out_path, "w") as outfile:
    for branch in BRANCH_LIST:
        branch_df = metadata_df[metadata_df["branch_minor"].isin(BRANCH_LIST[branch])]
        max_date = max(branch_df["date_mean"])
        outfile.write("{}\t{}\n".format(branch, max_date))
        mrsd_dict[branch] = max_date

## Colors

In [6]:
colors_dict = {}

colors_df = pd.read_csv(out_path_colors, sep='\t', header=None)
colors_df.columns = ["state", "value", "color"]

for state in set(colors_df["state"]):
    state_df = colors_df[colors_df["state"] == state]    
    colors_dict[state] = {}
    
    for value,color in zip(state_df["value"], state_df["color"]):
        colors_dict[state][value] = color

print(colors_dict)

{'branch_major': {'0.PRE': '#8000ff', '0.PE': '#4c4ffc', '0.ANT': '#1996f3', '3.ANT': '#1acee3', '4.ANT': '#4df3ce', '2.ANT': '#80ffb4', '2.MED': '#b3f396', '1.PRE': '#e6ce74', '1.ANT': '#ff964f', '1.IN': '#ff4f28', '1.ORI': '#ff0000', '?': '#969696'}, 'branch_minor': {'0.PRE2': '#8000ff', '0.PRE1': '#7018ff', '0.PE7': '#6130fe', '0.PE2': '#5148fc', '0.PE8': '#425ffa', '0.PE10': '#3275f8', '0.PE4': '#238af5', '0.PE5': '#139ef1', '0.ANT1': '#04b0ed', '0.ANT4': '#0cc1e8', '0.ANT5': '#1bd0e3', '0.ANT2': '#2bdddd', '0.ANT3': '#3ae8d7', '3.ANT1': '#49f1d0', '3.ANT2': '#59f8c8', '4.ANT1': '#68fcc1', '2.ANT3': '#78ffb9', '2.ANT2': '#87ffb0', '2.ANT1': '#97fca7', '2.MED0': '#a6f89e', '2.MED3': '#b6f194', '2.MED2': '#c5e88a', '2.MED1': '#d5dd7f', '1.PRE0': '#e4d075', '1.PRE1': '#f3c16a', '1.PRE2': '#ffb05f', '1.PRE3': '#ff9e53', '1.ANT1': '#ff8a48', '1.IN1': '#ff753c', '1.IN2': '#ff5f30', '1.IN3': '#ff4824', '1.ORI2': '#ff3018', '1.ORI1': '#ff180c', '1.ORI3': '#ff0000', '?': '#969696'}, 'provin

## Tree Files

In [7]:
# Construct a dictionary to hold the trees
tree_dict = {}

for branch in BRANCH_LIST:
    for filename in os.listdir(tree_dir):
        if not filename.endswith(".tre") and not filename.endswith(".tree"): continue         
        filepath = os.path.join(tree_dir, filename)
        
        # Option 1 when all trees are present
        #if branch in filename: 
        
        # Option 2 when not all trees are present
        if filename.split("_")[0] == branch:        
            print("Branch:", branch)
            tree_dict[branch] = {}    
            # Add tree files to dict
            tree_dict[branch]["tree_file_raw"] = filepath 
            tree_dict[branch]["tree_file_edit"] = os.path.join(tree_dir, branch + ".nex")
            tree_dict[branch]["sample_rename"] = {}
            
            # Read in raw tree to deal with dashes
            with open(tree_dict[branch]["tree_file_raw"],  "r") as infile:                    
                with open(tree_dict[branch]["tree_file_edit"], "w") as outfile:           
                    raw_tree = infile.read()
                    # Remove quotations if they exist
                    raw_tree = raw_tree.replace("'","")
                    
                    # Split into lines to iterate over
                    raw_tree_lines = raw_tree.split("\n")
                    
                    # By default don't parse a line for dashes
                    taxa_line = False
                    
                    for line in raw_tree_lines: 
                                
                        if "TREE" not in line and "-" in line:
                            
                            if len(line.split(" ")) == 1:
                                name_dashes = line.strip()
                                name_no_dashes = name_dashes.replace("-","_")
                                tree_dict[branch]["sample_rename"][name_no_dashes] = name_dashes
                            line = line.replace("-","_")
                                
                        outfile.write(line + "\n")
            
            # Read in edited tree
            trees = Phylo.parse(tree_dict[branch]["tree_file_edit"], "nexus")
            # There should be only 1 tree
            for t in trees:
                tree_dict[branch]["tree"] = t
                tree_dict[branch]["tree"].ladderize(reverse=False)
                break

            # Rename sample names back to with dashes
            for c in tree_dict[branch]["tree"].find_clades():
                if c.name in tree_dict[branch]["sample_rename"]:
                    orig_name = c.name
                    c.name = tree_dict[branch]["sample_rename"][c.name]
                    print("Rename:", orig_name, c.name)
                    
                # Strip the date suffix
                #if c.name:
                #    c.name = "_".join(c.name.split("_")[0:-1])           
                    
            # Rename internal nodes
            node_i = 0
            for c in tree_dict[branch]["tree"].find_clades():
                if not c.name:
                    c.name = "NODE{}".format(node_i)
                    node_i += 1

Branch: 1.ORI
Rename: GCA_000324805.2_EV76_CN_genomic GCA_000324805.2_EV76-CN_genomic
Rename: GCA_000986995.1_YPES001_SEQ_2_ASM_1_genomic GCA_000986995.1_YPES001-SEQ-2-ASM-1_genomic
Branch: 1.PRE
Branch: 2.MED
Rename: GCA_001617735.1_Yersinia_pestis_M_1763_genomic GCA_001617735.1_Yersinia_pestis_M-1763_genomic
Rename: GCA_002412305.1_Y.pestis_A_1809_genomic GCA_002412305.1_Y.pestis_A-1809_genomic
Rename: GCA_001617785.1_Yersinia_pestis_M_549_genomic GCA_001617785.1_Yersinia_pestis_M-549_genomic
Rename: GCA_001617725.1_Yersinia_pestis_M_1484_genomic GCA_001617725.1_Yersinia_pestis_M-1484_genomic
Rename: GCA_001617715.1_Yersinia_pestis_M_519_genomic GCA_001617715.1_Yersinia_pestis_M-519_genomic
Rename: GCA_001617815.1_Yersinia_pestis_M_1453_genomic GCA_001617815.1_Yersinia_pestis_M-1453_genomic
Rename: GCA_001617705.1_Yersinia_pestis_C_791_genomic GCA_001617705.1_Yersinia_pestis_C-791_genomic
Branch: 0.ANT4
Branch: 0.PRE


## Add Tree Data to Dataframe

### Get comments from the first root that is not root

In [8]:
TREE_PARAMETERS = None

for branch in tree_dict:
    
    print(branch)
    
    tree = tree_dict[branch]["tree"]
    df = copy.deepcopy(metadata_df[metadata_df["branch_minor"].isin(BRANCH_LIST[branch])])
    
    # Check if a sample was missed :(
    df_samples = df.index
    tree_samples = [c.name for c in tree.find_clades()]
    
    for sample in df_samples:
        if sample not in tree_samples:
            df.drop(sample, inplace=True)
            print("Dropping {} from the dataframe.".format(sample))
    
    root_comment_dict = parse_comment(tree.root.comment)
    
    for c in tree.find_clades():
        if c.is_terminal():
            continue
            
        if c != tree.root:
            comment_dict = parse_comment(c.comment)
            for parameter,value in comment_dict.items():
                # Intialize parameter values
                if "range" in parameter or "95%" in parameter:
                    df[parameter] = [[NO_DATA_CHAR,NO_DATA_CHAR]] * len(df)
                    if parameter not in root_comment_dict:
                        root_comment_dict[parameter] = '{0,0}'
                else:
                    df[parameter] = [NO_DATA_CHAR] * len(df)
                    if parameter not in root_comment_dict:
                        root_comment_dict[parameter] = '0'
                    
            break

    # Update the roots comment to include missing values
    new_comment = "[&"
    new_comment_list = []
    for parameter,value in root_comment_dict.items():
        param_str = "{}={}".format(parameter, value)
        new_comment_list.append(param_str)
    new_comment += ",".join(new_comment_list) + "]"
    tree.root.comment = new_comment
    #print(root_comment_dict)
    #print(new_comment_list)
    
    if not TREE_PARAMETERS:
        TREE_PARAMETERS = [p for p in root_comment_dict]
    
    tree_dict[branch]["df"] = df 
    #display(df)

1.ORI
1.PRE
2.MED
0.ANT4
0.PRE


### Parse Tree Comments

In [9]:
for branch in tree_dict:
    print(branch)
    
    tree = tree_dict[branch]["tree"]
    df = tree_dict[branch]["df"]
    
    # Initialize new columns
    df["timetree_num_date"] = [NO_DATA_CHAR] * len(df) # Calendar date of height
    df["timetree_num_date_confidence"] = [[NO_DATA_CHAR, NO_DATA_CHAR ]] * len(df)
    df["rate_sub_year"] = [NO_DATA_CHAR] * len(df)
    df["branch_length_sub"] = [NO_DATA_CHAR] * len(df)
    df["branch_support_conf_category"] = [NO_DATA_CHAR] * len(df)
    df["branch_support_conf_char"] = [NO_DATA_CHAR] * len(df)
    
    for c in tree.find_clades():
        
        # --------------------------------------
        # Get node type
        node_type = "internal"
        if c.is_terminal():
            node_type = "terminal"
        df.at[c.name, "node_type"] = node_type
        
        # --------------------------------------
        # Parse comments
        comment_dict = parse_comment(c.comment)
        for parameter,val in comment_dict.items():
            
            # Ranges should be parsed into list     
            if "range" in parameter or "95%" in parameter:
                val = [float(v) for v in val.strip("{}").split(",")]
            # Multiply the posterior by 100 to be comparable to UFboot
            elif parameter == "posterior":
                val = float(val) * 100
            # Make sure states are strings
            elif parameter == "state":
                parameter = "country"
                val = str(val) 
            elif parameter == "state.set":
                val = str(val)                  
            elif parameter == "state.set.prob":
                val = str(val)
            elif parameter == "default.rate":
                parameter = "rate"
                val = float(val)
            else:
                val = float(val)
            df.at[c.name, parameter] = val
            
            
        # Tips don't have posteriors, set to 0
        if c.is_terminal():
            posterior = 0
            df.at[c.name, "posterior"] = posterior
            comment_dict["posterior"] = posterior
            
        # Set confidence category for posterior/branch support
        posterior = df["posterior"][c.name]
        branch_support_conf_category = "LOW"
        branch_support_conf_char = ""
        if posterior >= POSTERIOR_THRESH:
            branch_support_conf_category = "HIGH"
            branch_support_conf_char = "*"
        df.at[c.name,"branch_support_conf_category"] = branch_support_conf_category
        df.at[c.name,"branch_support_conf_char"] = branch_support_conf_char

        # Check for missing parameters
        # For example, the most recent sample will be missing height uncertainty
        for parameter in TREE_PARAMETERS:
            if parameter not in comment_dict:
                if "range" in parameter or "95%" in parameter:
                    val = [0,0]
                else:
                    val = 0
                df.at[c.name, parameter] = val
                comment_dict[parameter] = val
                    
                #print(c.name, parameter, df["date"][c.name], mrsd_dict[branch])
            
        # Convert height to calendar dates
        height = df["height"][c.name]
        height_95_hpd = df["height_95%_HPD"][c.name]
        
        # The sample with the most recent date does not have a height 95% HPD
        if height_95_hpd[0] == NO_DATA_CHAR:
            height_95_hpd = [0,0]
            df.at[c.name, "height_95%_HPD"] = height_95_hpd
            
        height_calendar = mrsd_dict[branch] - height
        height_95_hpd_calendar = [mrsd_dict[branch] -  height_95_hpd[1], mrsd_dict[branch] -  height_95_hpd[0]]
    
        df.at[c.name, "timetree_num_date"] = height_calendar
        df.at[c.name, "timetree_num_date_confidence"] = height_95_hpd_calendar
        
        # Add another measure of substitution rate
        rate_sub_year = df["rate"][c.name] * SEQ_LEN
        df.at[c.name, "rate_sub_year"] = rate_sub_year
        
        # Measure branch length in substitutions
        branch_length_sub = rate_sub_year * df["length"][c.name]
        df.at[c.name, "branch_length_sub"] = branch_length_sub
    
    df.fillna(NO_DATA_CHAR, inplace=True)

1.ORI
1.PRE
2.MED
0.ANT4
0.PRE


## Reduced Dataframe for Auspice

In [10]:
# OPtions
print(tree_dict[branch]["df"].columns)

Index(['strain', 'date', 'date_bp', 'country', 'province', 'country_lat',
       'country_lon', 'province_lat', 'province_lon', 'biovar', 'branch_major',
       'branch_minor', 'biosample_accession', 'biosample_comment',
       'branch_number', 'continent', 'date_mean', 'date_bp_mean', 'date_err',
       'lat', 'lon', 'host_human', 'branch_major_color', 'geometry_size',
       'geometry', 'root_rtt_dist', 'clade_rtt_dist', 'length_range',
       'state.rate_range', 'default.rate_95%_HPD', 'length_95%_HPD',
       'state.rate_95%_HPD', 'default.rate_range', 'state.rate',
       'default.rate', 'length', 'posterior', 'height_median', 'height_range',
       'height_95%_HPD', 'state.rate_median', 'default.rate_median',
       'length_median', 'height', 'timetree_num_date',
       'timetree_num_date_confidence', 'rate_sub_year', 'branch_length_sub',
       'branch_support_conf_category', 'branch_support_conf_char', 'node_type',
       'rate', 'state.prob', 'state', 'state.set.prob', 'state.

In [11]:
for branch in tree_dict:
    print(branch)
    
    columns = [
        # Node type is mandatorya
        "node_type",        
        # Draw Divergence Tree
        "length",
        # Draw Time Tree
        "timetree_num_date",
        "timetree_num_date_confidence",  
        # Geo
        "country",
        "province",
        "country_lat",
        "country_lon",
        "province_lat",
        "province_lon",           
        # Stats
        "posterior",
        "branch_support_conf_category",
        "branch_support_conf_char",
        "rate",
        "rate_sub_year",
        "branch_length_sub",
        # Text descriptions
        "strain",
        "branch_major",
        "biovar",
        "host_human",
        "date_mean",
        "date_bp_mean",        
    ]
    
    auspice_df = copy.copy(tree_dict[branch]["df"][columns])
    
    # Round For Pretty Numbers
    posteriors = [float(round(p)) for p in list(auspice_df["posterior"])]
    auspice_df["posterior"] = posteriors  
    
    branch_length_times = [float(round(l)) for l in list(auspice_df["length"])] 
    auspice_df["branch_length_time"] = branch_length_times
    
    branch_length_subs = [float(round(l)) for l in list(auspice_df["branch_length_sub"])] 
    auspice_df["branch_length_sub"] = branch_length_subs 

    rate_sub_years = [float(round(s)) for s in list(auspice_df["rate_sub_year"])] 
    auspice_df["rate_sub_year"] = rate_sub_years
    
    auspice_df.rename(columns={"length": "branch_length"}, inplace=True)
    auspice_df.rename(columns={"posterior": "branch_support"}, inplace=True)
    auspice_df.rename(columns={"rate": "rate_sub"}, inplace=True)
    
    # Add blank column
    auspice_df["blank"] = [" "] * len(auspice_df)

    
    tree_dict[branch]["auspice_df"] = auspice_df
    display(tree_dict[branch]["auspice_df"])
    

1.ORI


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,country,province,country_lat,country_lon,province_lat,province_lon,branch_support,branch_support_conf_category,branch_support_conf_char,rate_sub,rate_sub_year,branch_length_sub,strain,branch_major,biovar,host_human,date_mean,date_bp_mean,branch_length_time,blank
sample,,,,,,,,,,,,,,,,,,,,,,,,
Reference,terminal,14.290093,1992.000000,"[1991.999999999996, 1992.000000000004]",United States of America,Colorado,39.7837,-100.446,38.7252,-105.608,0.0,LOW,,4.231324e-07,2.0,26.0,CO92,1.ORI,Orientalis,Human,1992,29,14.0,
GCA_000834775.1_ASM83477v1_genomic,terminal,6.975785,1967.000000,"[1966.999999999993, 1967.00000000001]",United States of America,Arizona,39.7837,-100.446,34.3953,-111.763,0.0,LOW,,3.163987e-07,1.0,9.0,Dodson,1.ORI,Orientalis,Human,1967,54,7.0,
GCA_000834335.1_ASM83433v1_genomic,terminal,4.681411,1954.000000,"[1953.9999999999918, 1954.0000000000082]",United States of America,California,39.7837,-100.446,36.7015,-118.756,0.0,LOW,,2.270539e-07,1.0,4.0,Shasta,1.ORI,Orientalis,Human,1954,67,5.0,
GCA_000169635.1_ASM16963v1_genomic,terminal,27.879200,2005.000000,"[2004.9999999999675, 2005.00000000003]",Madagascar,NA,-18.925,46.4416,NA,NA,0.0,LOW,,1.330186e-07,1.0,16.0,MG05-1020,1.ORI,Orientalis,Human,2005,16,28.0,
GCA_000170275.1_ASM17027v1_genomic,terminal,14.388214,1991.000000,"[1990.999999999984, 1991.000000000016]",China,Yunnan,35.0001,105,25,102,0.0,LOW,,8.240116e-08,0.0,5.0,F1991016,1.ORI,Orientalis,Non-Human,1991,30,14.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE111,internal,1.850724,2004.741212,"[2001.2031002225738, 2007.743490769209]",Peru,NA,NA,NA,NA,NA,24.0,LOW,,1.684506e-07,1.0,1.0,NA,NA,NA,NA,NA,NA,2.0,
NODE112,internal,2.986264,2008.107821,"[2005.6455957831777, 2009.904784379414]",Peru,NA,NA,NA,NA,NA,96.0,HIGH,*,1.382195e-07,1.0,2.0,NA,NA,NA,NA,NA,NA,3.0,
NODE113,internal,1.381419,2006.159421,"[2003.379419273439, 2008.5135808224536]",Peru,NA,NA,NA,NA,NA,41.0,LOW,,2.024896e-07,1.0,1.0,NA,NA,NA,NA,NA,NA,1.0,


1.PRE


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,country,province,country_lat,country_lon,province_lat,province_lon,branch_support,branch_support_conf_category,branch_support_conf_char,rate_sub,rate_sub_year,branch_length_sub,strain,branch_major,biovar,host_human,date_mean,date_bp_mean,branch_length_time,blank
sample,,,,,,,,,,,,,,,,,,,,,,,,
SAMEA5818830,terminal,2.258639,1560.000000,"[1559.9999999999498, 1560.0000000000514]",Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,0.0,LOW,,3.439042e-08,0.0,0.0,STN021,1.PRE,Second Pandemic,Human,1560,461,2.0,
SAMEA5818829,terminal,2.972765,1560.000000,"[1559.99999999995, 1560.0000000000514]",Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,0.0,LOW,,3.368854e-08,0.0,0.0,STN020,1.PRE,Second Pandemic,Human,1560,461,3.0,
SAMEA5818828,terminal,2.951818,1560.000000,"[1559.99999999995, 1560.0000000000514]",Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,0.0,LOW,,3.335237e-08,0.0,0.0,STN019,1.PRE,Second Pandemic,Human,1560,461,3.0,
SAMEA5818826,terminal,3.024750,1560.000000,"[1559.9999999999498, 1560.0000000000514]",Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,0.0,LOW,,3.443847e-08,0.0,0.0,STN014,1.PRE,Second Pandemic,Human,1560,461,3.0,
SAMEA5818825,terminal,2.258639,1560.000000,"[1559.9999999999498, 1560.0000000000514]",Switzerland,Nidwalden,46.8133,8.44495,46.9428,8.41198,0.0,LOW,,1.172794e-07,0.0,1.0,STN013,1.PRE,Second Pandemic,Human,1560,461,2.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE34,internal,2.526181,1552.856734,"[1547.343805771212, 1557.7926399457879]",NA,NA,NA,NA,NA,NA,13.0,LOW,,3.700155e-08,0.0,0.0,NA,NA,NA,NA,NA,NA,3.0,
NODE35,internal,2.560544,1554.070167,"[1548.899359250475, 1558.5969363982726]",NA,NA,NA,NA,NA,NA,6.0,LOW,,3.733455e-08,0.0,0.0,NA,NA,NA,NA,NA,NA,3.0,
NODE36,internal,3.015103,1556.055502,"[1551.3016283684117, 1559.5395201477922]",NA,NA,NA,NA,NA,NA,9.0,LOW,,3.328841e-08,0.0,0.0,NA,NA,NA,NA,NA,NA,3.0,


2.MED


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,country,province,country_lat,country_lon,province_lat,province_lon,branch_support,branch_support_conf_category,branch_support_conf_char,rate_sub,rate_sub_year,branch_length_sub,strain,branch_major,biovar,host_human,date_mean,date_bp_mean,branch_length_time,blank
sample,,,,,,,,,,,,,,,,,,,,,,,,
GCA_009909635.1_ASM990963v1_genomic,terminal,19.016925,1923.000000,"[1922.9999999999575, 1923.000000000043]",Russia,Rostov Oblast,64.6863,97.7453,47.6222,40.7958,0.0,LOW,,1.699646e-07,1.0,14.0,9_10,2.MED,Medievalis,Human,1923,98,19.0,
GCA_009296005.1_ASM929600v1_genomic,terminal,12.849306,1953.000000,"[1952.9999999999568, 1953.0000000000425]",Russia,Chechnya,64.6863,97.7453,43.3976,45.6985,0.0,LOW,,7.077325e-08,0.0,4.0,C-25,2.MED,Medievalis,Non-Human,1953,68,13.0,
GCA_008630485.1_ASM863048v1_genomic,terminal,3.617000,1997.000000,"[1996.9999999999566, 1997.0000000000427]",Russia,Kabardino-Balkaria,64.6863,97.7453,43.4428,43.4205,0.0,LOW,,6.573191e-07,3.0,10.0,C-742,2.MED,Medievalis,Non-Human,1997,24,4.0,
GCA_008630435.1_ASM863043v1_genomic,terminal,24.303254,1996.000000,"[1995.999999999986, 1996.0000000000118]",Russia,Karachay-Cherkessia,64.6863,97.7453,43.7368,41.7268,0.0,LOW,,6.572837e-07,3.0,68.0,C-719,2.MED,Medievalis,Non-Human,1996,25,24.0,
GCA_008630395.1_ASM863039v1_genomic,terminal,8.228695,1984.000000,"[1983.999999999955, 1984.0000000000412]",Russia,Republic of Dagestan,64.6863,97.7453,43.0883,47.1499,0.0,LOW,,3.738636e-07,2.0,13.0,C-528,2.MED,Medievalis,Non-Human,1984,37,8.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE110,internal,8.987725,1989.038486,"[1982.178482146981, 1995.4930958935863]",NA,NA,NA,NA,NA,NA,100.0,HIGH,*,1.284091e-07,1.0,5.0,NA,NA,NA,NA,NA,NA,9.0,
NODE111,internal,3.602935,1993.383000,"[1988.5689248739607, 1996.8665954433511]",NA,NA,NA,NA,NA,NA,100.0,HIGH,*,1.520278e-07,1.0,2.0,NA,NA,NA,NA,NA,NA,4.0,
NODE112,internal,9.115426,1998.339317,"[1988.7757888771998, 2007.4732478904593]",NA,NA,NA,NA,NA,NA,73.0,LOW,,2.652168e-08,0.0,1.0,NA,NA,NA,NA,NA,NA,9.0,


0.ANT4


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,country,province,country_lat,country_lon,province_lat,province_lon,branch_support,branch_support_conf_category,branch_support_conf_char,rate_sub,rate_sub_year,branch_length_sub,strain,branch_major,biovar,host_human,date_mean,date_bp_mean,branch_length_time,blank
sample,,,,,,,,,,,,,,,,,,,,,,,,
SAMEA5661390,terminal,5.934884,478.000000,"[477.9999999999499, 478.0000000000501]",Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,0.0,LOW,,3.415710e-06,14.0,86.0,UNT004,0.ANT,Antiqua,Human,478,1543,6.0,
SAMEA5661389,terminal,18.106782,516.000000,"[515.9999999999488, 516.0000000000514]",Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,0.0,LOW,,3.624344e-09,0.0,0.0,UNT003,0.ANT,Antiqua,Human,516,1505,18.0,
SAMEA5661385,terminal,18.990485,517.000000,"[516.99999999995, 517.00000000005]",Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,0.0,LOW,,6.547866e-07,3.0,53.0,PET004,0.ANT,Antiqua,Human,517,1504,19.0,
SAMEA5661384,terminal,86.679955,521.000000,"[520.99999999994, 521.00000000006]",Spain,Valencia Community,39.3261,-4.83798,39.682,-0.765441,0.0,LOW,,1.792881e-08,0.0,7.0,VAL001,0.ANT,Antiqua,Human,521,1500,87.0,
SAMEA5661372,terminal,47.054375,765.000000,"[765.0, 765.0]",France,Centre-Loire Valley,46.6034,1.88833,47.549,1.73241,0.0,LOW,,1.268521e-07,1.0,25.0,LSD020,0.ANT,Antiqua,Human,765,1256,47.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE6,internal,39.631380,500.919910,"[474.467398194792, 516.7926070135841]",Germany,NA,NA,NA,NA,NA,93.0,LOW,,2.400574e-09,0.0,0.0,NA,NA,NA,NA,NA,NA,40.0,
NODE7,internal,220.065731,717.945625,"[637.2782153708, 764.6461654102749]",France,NA,NA,NA,NA,NA,100.0,HIGH,*,2.018236e-08,0.0,19.0,NA,NA,NA,NA,NA,NA,220.0,
NODE8,internal,9.737297,471.254535,"[458.37663796541597, 477.97856227659]",Germany,NA,NA,NA,NA,NA,100.0,HIGH,*,1.221871e-07,1.0,5.0,NA,NA,NA,NA,NA,NA,10.0,


0.PRE


,node_type,branch_length,timetree_num_date,timetree_num_date_confidence,country,province,country_lat,country_lon,province_lat,province_lon,branch_support,branch_support_conf_category,branch_support_conf_char,rate_sub,rate_sub_year,branch_length_sub,strain,branch_major,biovar,host_human,date_mean,date_bp_mean,branch_length_time,blank
sample,,,,,,,,,,,,,,,,,,,,,,,,
SAMEA104233048,terminal,140.358180,-1944.500000,"[-1944.5, -1944.5]",Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,0.0,LOW,,1.668537e-08,0.0,10.0,Post6,0.PRE,Bronze Age,Human,-1944.5,3965.5,140.0,
SAMEA104233049,terminal,11.894365,-2272.000000,"[-2272.00000000008, -2271.999999999927]",Germany,Bavaria,51.0834,10.4234,48.9468,11.4039,0.0,LOW,,1.706801e-06,7.0,86.0,1343UnTal85,0.PRE,Bronze Age,Human,-2272,4293,12.0,
SAMEA104233047,terminal,244.138160,-2457.000000,"[-2457.00000000011, -2456.99999999989]",Estonia,Pärnu maakond,58.7524,25.3319,58.3195,24.3026,0.0,LOW,,1.668537e-08,0.0,17.0,KunilaII,0.PRE,Bronze Age,Human,-2457,4478,244.0,
SAMEA104233046,terminal,198.228985,-2546.500000,"[-2546.500000000111, -2546.499999999885]",Lithuania,Panevezys County,55.35,23.75,55.9156,25.0312,0.0,LOW,,1.668537e-08,0.0,14.0,Gyvakarai1,0.PRE,Bronze Age,Human,-2546.5,4567.5,198.0,
SAMEA3541827,terminal,33.455487,-2776.500000,"[-2776.5000000001, -2776.499999999893]",Russia,Krasnoyarsk Krai,64.6863,97.7453,63.3234,97.098,0.0,LOW,,1.518149e-06,6.0,215.0,RISE509,0.PRE,Bronze Age,Human,-2776.5,4797.5,33.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE2,internal,47.574086,-2700.108352,"[-2748.16251438543, -2668.2079570299275]",NA,NA,NA,NA,NA,NA,99.0,HIGH,*,1.337582e-06,6.0,269.0,NA,NA,NA,NA,NA,NA,48.0,
NODE3,internal,13.587994,-2686.600603,"[-2729.2763547802, -2666.6124803596003]",NA,NA,NA,NA,NA,NA,99.0,HIGH,*,6.735803e-07,3.0,39.0,NA,NA,NA,NA,NA,NA,14.0,
NODE4,internal,16.606139,-2671.275464,"[-2682.524693830586, -2666.500584737746]",NA,NA,NA,NA,NA,NA,100.0,HIGH,*,7.313060e-07,3.0,51.0,NA,NA,NA,NA,NA,NA,17.0,


---

# Export

## Time Tree

In [12]:
for branch in tree_dict:
    out_timetree = copy.deepcopy(tree_dict[branch]["tree"])

    metadata_to_comment(out_timetree, tree_dict[branch]["df"])    
    out_timetree_nex_path = os.path.join(tree_dir, branch + ".timetree.nex")
    Phylo.write(out_timetree, out_timetree_nex_path, "nexus")

    for c in out_timetree.find_clades():
        c.comment = None

    out_timetree_nwk_path = os.path.join(tree_dir, branch + ".timetree.nwk")
    Phylo.write(out_timetree, out_timetree_nwk_path, "newick")

## Augur

In [13]:
for branch in tree_dict:
    
    augur_dict = augur_export(
        tree_path=None, 
        aln_path=None,  
        tree=tree_dict[branch]["tree"], 
        tree_df=tree_dict[branch]["auspice_df"], 
        color_keyword_exclude=["geometry"],
        type_convert = {
            "branch_number" : (lambda x : str(x))
        },
    )
    
    tree_dict[branch]["augur_dict"] = augur_dict
    
    first_taxa = list(augur_dict["nodes"].keys())[0]
    print(augur_dict["nodes"][first_taxa])

    out_path_augur_json = os.path.join(tree_dir, branch + "_augur.json" )
    utils.write_json(data=tree_dict[branch]["augur_dict"], file_name=out_path_augur_json, indent=JSON_INDENT)
    tree_dict[branch]["augur_json_path"] = out_path_augur_json
    

{'node_type': 'internal', 'branch_length': 0.0, 'num_date': 1867.0942100091504, 'num_date_confidence': [1811.2040243167871, 1907.3711522232752], 'country': 'China', 'province': 'NA', 'country_lat': 'NA', 'country_lon': 'NA', 'province_lat': 'NA', 'province_lon': 'NA', 'branch_support': 100.0, 'branch_support_conf_category': 'HIGH', 'branch_support_conf_char': '*', 'rate_sub': 0.0, 'rate_sub_year': 0.0, 'branch_length_sub': 0.0, 'strain': 'NA', 'branch_major': 'NA', 'biovar': 'NA', 'host_human': 'NA', 'date_mean': 'NA', 'date_bp_mean': 'NA', 'branch_length_time': 0.0, 'blank': ' '}
{'node_type': 'internal', 'branch_length': 0.0, 'num_date': 1274.5194429677265, 'num_date_confidence': [1215.4213839601462, 1318.87619038616], 'country': 'NA', 'province': 'NA', 'country_lat': 'NA', 'country_lon': 'NA', 'province_lat': 'NA', 'province_lon': 'NA', 'branch_support': 100.0, 'branch_support_conf_category': 'HIGH', 'branch_support_conf_char': '*', 'rate_sub': 0.0, 'rate_sub_year': 0.0, 'branch_len

## Auspice

In [14]:
for branch in tree_dict:
    
    # Store the color
    if branch == "0.ANT4":
        branch_major_color = colors_dict["branch_major"]["0.ANT"]
    else:
        branch_major_color = colors_dict["branch_major"][branch]

    auspice_dict = auspice_export(
        tree=tree_dict[branch]["tree"],
        augur_json_paths=tree_dict[branch]["augur_json_path"], 
        auspice_config_path=auspice_config_path, 
        auspice_colors_path=out_path_colors,
        auspice_latlons_path=out_path_latlon, 
        )


    label_col = list(tree_dict[branch]["auspice_df"])
    print(label_col)

    # Recursively add branch attrs
    branch_attributes(
        tree_dict=auspice_dict["tree"], 
        sub_dict=auspice_dict["tree"], 
        df=tree_dict[branch]["auspice_df"],
        label_col=label_col,
        )
    
    
    # Last manual changes
    auspice_dict_copy = copy.deepcopy(auspice_dict)
    for i in range(0, len(auspice_dict_copy["meta"]["colorings"])):
        coloring = auspice_dict_copy["meta"]["colorings"][i]
        for key in coloring:
            # Node type as internal or terminal
            if coloring[key] == "node_type":
                auspice_dict["meta"]["colorings"][i]['scale'] = [['internal', '#FFFFFF'], ['terminal', branch_major_color]]
                #print(auspice_dict["meta"]["colorings"][i])
            # Confidence category
            if "conf_category" in coloring[key]:
                auspice_dict["meta"]["colorings"][i]['scale'] = [['LOW', '#FFFFFF'], ['HIGH', branch_major_color]]
                #print(auspice_dict["meta"]["colorings"][i])
            # Host Human binary
            if "host_human" in coloring[key]:
                auspice_dict["meta"]["colorings"][i]['scale'] = [['Human', '#CBB742'], ['Non-Human', "#60B6F2"], ['NA', "#D6D6D6"]]

    # Write outputs - For Local Rendering
    out_path_auspice_local_json = os.path.join(tree_dir, branch + ".json" )
    utils.write_json(data=auspice_dict, file_name=out_path_auspice_local_json, indent=JSON_INDENT, include_version=False)
    export_v2.validate_data_json(out_path_auspice_local_json)
    print("Validation successful for local JSON.\n")

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.
['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'country', 'province', 'country_lat', 'country_lon', 'province_lat', 'province_lon', 'branch_support', 'branch_support_conf_category', 'branch_support_conf_char', 'rate_sub', 'rate_sub_year', 'branch_length_sub', 'strain', 'branch_major', 'biovar', 'host_human', 'date_mean', 'date_bp_mean', 'branch_length_time', 'blank']


Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/phylogeography/1.ORI.json'...


Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/phylogeography/1.ORI.json' succeeded, but there were warnings you may want to resolve.
Validation successful for local JSON.

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.
['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'country', 'province', 'country_lat', 'country_lon', 'province_lat', 'province_lon', 'branch_support', 'branch_support_conf_category', 'branch_support_conf_char', 'rate_sub', 'rate_sub_year', 'branch_length_sub', 'strain', 'branch_major', 'biovar', 'host_human', 'date_mean', 'date_bp_mean', 'branch_length_time', 'blank']
Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/phylogeography/1.PRE.json'...
Val

Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/phylogeography/2.MED.json'...


Validating that the JSON is internally consistent...
Validation of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/phylogeography/2.MED.json' succeeded, but there were warnings you may want to resolve.
Validation successful for local JSON.

Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/config/auspice_config.json'...
Validation success.
['node_type', 'branch_length', 'timetree_num_date', 'timetree_num_date_confidence', 'country', 'province', 'country_lat', 'country_lon', 'province_lat', 'province_lon', 'branch_support', 'branch_support_conf_category', 'branch_support_conf_char', 'rate_sub', 'rate_sub_year', 'branch_length_sub', 'strain', 'branch_major', 'biovar', 'host_human', 'date_mean', 'date_bp_mean', 'branch_length_time', 'blank']
Validating produced JSON
Validating schema of '/mnt/c/Users/ktmea/Projects/plague-phylogeography-projects/main/beast/all/chromosome/clade/phylogeography/0.ANT4.json'...
Va